In [ ]:
from selenium import webdriver
import numpy as np
import pandas as pd

In [ ]:
import csv

# For S&P500 index, read all the data first: (as for RLV, RUS and other indices, just change the path)
new_list = []
with open("./index company/S&P.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        new_list.append(row)

# Re-organize the data read in the form of dataframe---cr_disclosure_company stores the result
new_list = np.array(new_list).T
new_dict = {}
for i in range(3):
    new_dict[new_list[i][0]] = new_list[i][1:]
cr_disclosure_company = pd.DataFrame(new_dict)
cr_disclosure_company = cr_disclosure_company[~cr_disclosure_company["ENVIRON_DISCLOSURE_SCORE"].isin(["#N/A N/A"])]

In [ ]:
# The followig script is for automatic sustainability report downloading from SDD-GDI database,
# Ryan you can consider to define an independent class for this part

used_company_names = [name.lower() for name in list(cr_disclosure_company["Company name"])]

# Preprocessing for search engine in SDD-GRI Database
search_company_names = []
annoying_tails = ["inc", "inc/the", "corp", "co/the", "co", "plc", "ltd", "nv"]
for name in used_company_names:
    list_name = name.split(" ")
    # We need to remove the 'annoying talis' in the company name we are going to type in the search engine
    # So that we can find what we want from the searching result
    if list_name[-1] in annoying_tails:
        search_company_names.append(" ".join(list_name[:-1]))
    else:
        search_company_names.append(" ".join(list_name))
        
# Selenium auto searching in SDD-GRI Database
pdf_2018_url_list=[]    
for name in search_company_names:
    # Start the webdriver
    browser = webdriver.Chrome()
    # Switch to the website
    browser.get("https://database.globalreporting.org/")
    # Search the element in the website
    search_box = browser.find_element_by_name("q")
    
    # use try-except structure to avoid circumstances where no results being found
    try:
        # Use search engine in SDD-GRI to get to the company we want
        search_box.send_keys(name)
        search_box.submit()
        # Only search for reports published after 2010
        elem = browser.find_element_by_partial_link_text("201")
        elem.click()
        # Search for PDF we want
        elem1 = browser.find_element_by_partial_link_text("PDF")
        elem1.click()
        # Save the pdf link
        pdf_2018_url_list.append(browser.current_url)
        browser.close()
    except:
        browser.close()
        continue

# Result
pdf_2018_url_list

# PS: We have url list but we cannot use it to automatic download these reports directly because:
# some anti-spydering mechanism like reCAPTCHA will block our automatic downloading process within SDD-GRI website
# Future work could be done from what we have achieved above

['https://database.globalreporting.org/reports/78447/',
 'https://database.globalreporting.org/reports/65284/',
 'https://database.globalreporting.org/reports/60414/',
 'https://database.globalreporting.org/reports/58505/',
 'https://database.globalreporting.org/reports/60850/',
 'https://database.globalreporting.org/reports/60850/',
 'https://database.globalreporting.org/reports/64927/',
 'https://database.globalreporting.org/reports/35244/',
 'https://database.globalreporting.org/reports/73939/',
 'https://database.globalreporting.org/reports/73592/',
 'https://database.globalreporting.org/reports/62044/',
 'https://database.globalreporting.org/reports/77970/',
 'https://database.globalreporting.org/reports/65580/',
 'https://database.globalreporting.org/reports/73915/',
 'https://database.globalreporting.org/reports/37254/',
 'https://database.globalreporting.org/reports/75982/',
 'https://database.globalreporting.org/reports/75444/',
 'https://database.globalreporting.org/reports/1

In [ ]:
# This is data for RLV index, same as above for S&P500

rlv_list = []
with open("./index company/RLV.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        rlv_list.append(row)

new_list = np.array(rlv_list).T
new_dict = {}
for i in range(3):
    new_dict[new_list[i][0]] = new_list[i][1:]
rlv_cr_disclosure_company = pd.DataFrame(new_dict)
rlv_cr_disclosure_company = rlv_cr_disclosure_company[~rlv_cr_disclosure_company["ENV_DISCOSURE_SCORE"].isin(["#N/A N/A"])]
rlv_cr_disclosure_company

,Name,ENV_DISCOSURE_SCORE,company name
1,AA UN,57.36434109,Alcoa Corp
3,AAP UN,22.48062016,Advance Auto Parts Inc
6,ACC UN,4.651162791,American Campus Communities Inc
7,ACGL UW,1.785714286,Arch Capital Group Ltd
8,ACHC UW,6.201550388,Acadia Healthcare Co Inc
...,...,...,...
742,WTFC UW,3.571428571,Wintrust Financial Corp
746,WWD UW,16.27906977,Woodward Inc
750,X UN,13.95348837,United States Steel Corp
759,YUMC UN,32.29166667,Yum China Holdings Inc
